In [193]:

# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json
from datetime import datetime, timedelta
import random
import requests
from langdetect import detect
from urllib.parse import urljoin
import os
import numpy as np


In [194]:
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [195]:
def get_company_name(soup):
    # Tìm thẻ h1 với class mong muốn
    company_name_tag = soup.find('h1', class_='sc-ca95509a-8 gcvyPj')
    # Kiểm tra nếu tìm thấy thẻ, trả về nội dung text
    if company_name_tag:
        return company_name_tag.get_text(strip=True)
    return None


In [196]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return canonical_url
    return None

In [197]:
def get_id_cong_ty(soup):
    url = get_url(soup)
    # Biểu thức chính quy tìm kiếm mã ID sau 'c'
    match = re.search(r'c(\d+)', url)
    if match:
        return match.group(1)  # Trả về kết quả khớp hoàn toàn 'c311795'
    return None

In [198]:
def get_company_info(soup):
    result = {"quyMo": None, "linhVuc": None, "lienHe": None}
    li_tags = soup.find_all('li', class_='sc-7f4c261d-6 ejuuLs')

    for li in li_tags:
        type_text = li.find('p', class_='type').get_text(strip=True)
        if type_text.lower() == "quy mô":
            result["quyMo"] = str(li.find('p', class_='text').get_text(strip=True))
        elif type_text.lower() == "lĩnh vực":
            result["linhVuc"] = str(li.find('p', class_='text').get_text(strip=True))
        elif type_text.lower() == "liên hệ":
            result["lienHe"] = str(li.find('p', class_='text').get_text(strip=True))
    # result["diaChi"] = soup.find('div', class_='dangerously-text').get_text(strip=True)
    return result


In [199]:
def get_logo(soup):
    try:
        # Tìm tag chứa ảnh logo
        style = soup.find("div", class_="sc-ca95509a-3 fbLezD").find("div").get('style')
        if not style:
            return None
        
        # Lấy link logo
        logo_url = re.search(r"url\('(.+?)'\)", style).group(1)
        output_folder="logos_vietnamworks"
        logo_url = urljoin('https://images.vietnamworks.com', logo_url)  # Hoàn thiện URL
        # print(logo_url)
        # Lấy id công ty
        company_id = get_id_cong_ty(soup)  # Hàm này do bạn cung cấp
        
        # Tạo tên file và thư mục lưu
        filename = f"logo_{company_id}.jpg"
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, filename)
        
        # Tải ảnh từ link
        response = requests.get(logo_url, stream=True)
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        return logo_url
    
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [200]:
def get_dia_chi(soup):
    # Tìm thẻ <script> có id="__NEXT_DATA__"
    script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})
    
    if script_tag:
        # Parse nội dung JSON trong thẻ <script>
        try:
            data = json.loads(script_tag.string)
            company_info = data.get('props', {}).get('pageProps', {}).get('companyProfile', {}).get('address')
            return company_info
        except json.JSONDecodeError:
            print("Không thể phân tích JSON.")
            return None
    else:
        print("Không tìm thấy thẻ <script> chứa JSON.")
        return None

In [201]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [202]:
def get_phuc_loi(soup):
    result = []
    data = soup.find_all('div', class_="sc-eifrsQ huJZbu")
    
    for temp in data:
        # Tìm thẻ h4 (key)
        obj = temp.find('h4')
        # Tìm thẻ div chứa value
        value = temp.find('div', class_="sc-d6a527f2-6 gsnZik")
        
        # Lấy nội dung text nếu tồn tại
        obj_text = obj.get_text(strip=True) if obj else None
        value_text = value.get_text(strip=True) if value else None
        
        # Thêm vào từ điển nếu cả obj và value tồn tại
        if obj_text and value_text:
            # Thêm đối tượng theo cấu trúc yêu cầu vào result
            result.append({
                "Name": obj_text,
                "Description": value_text
            })
    
    return result


In [203]:
def get_gioi_thieu(soup):
    # Tìm thẻ <script> có id="__NEXT_DATA__"
    script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})
    
    if script_tag:
        # Parse nội dung JSON trong thẻ <script>
        try:
            data = json.loads(script_tag.string)
            company_info = data.get('props', {}).get('pageProps', {}).get('companyProfile', {}).get('companyProfile')
            return company_info
        except json.JSONDecodeError:
            print("Không thể phân tích JSON.")
            return None
    else:
        print("Không tìm thấy thẻ <script> chứa JSON.")
        return None

In [204]:
# output = []
# for temp in range(0,100):
# # for temp in UrlCongTy:
#     while True:
#         try:
#             ids = 4018 + temp
#             scraper = cloudscraper.create_scraper(
#                             browser={
#                                 "browser": "chrome",
#                                 "platform": "windows",
#                             },
#                         )
#             # specify the target URL
#             url = f"https://www.vietnamworks.com/nha-tuyen-dung/abc-solutions-c{ids}?fromPage=jobDetail"
#             # url = temp
#             response = scraper.get(url)
#             print(response.status_code, ids)
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.text, 'html.parser')
#                 if get_id_cong_ty(soup):
#                     data = {
#                         'Url': str(get_url(soup) + "?fromPage=jobDetail"),
#                         'ID': int(get_id_cong_ty(soup)),
#                         'Logo': str(get_logo(soup)),
#                         'Name': str(get_company_name(soup)),
#                         'CompanySize': get_company_info(soup).get('quyMo'),
#                         'Field': get_company_info(soup).get('linhVuc'),
#                         'Contact': get_company_info(soup).get('lienHe'),
#                         'CompanyIntroduction': str(get_gioi_thieu(soup)),
#                         'CompanyAddress': str(get_dia_chi(soup)),
#                         'Welfare': get_phuc_loi(soup),
#                     }
#                     output.append(data)
#                     print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))
#                     print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
#             if response.status_code == 429:
#                     # print(f"ID {ids}: Too many requests, retrying...")
#                 time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
#             else:
#                 break  # Thoát khỏi vòng lặp nếu không phải 429
#         except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
#             print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
#             time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại


In [205]:
# Đường dẫn tới file JSON
file_path = 'sample_vietnamworks_28_11_2024_2.json'

# Đọc file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Hiển thị dữ liệu đã đọc
# print(data)
UrlCongTy = []
for temp in data:
    UrlCongTy.append(temp.get('UrlCongTy'))


In [206]:
output = []
# for temp in range(0,100):
for temp in UrlCongTy:
    while True:
        try:
            # ids = 299405 + temp
            scraper = cloudscraper.create_scraper(
                            browser={
                                "browser": "chrome",
                                "platform": "windows",
                            },
                        )
            # specify the target URL
            # url = f"https://www.vietnamworks.com/nha-tuyen-dung/abc-solutions-c{ids}?fromPage=jobDetail"
            url = temp
            response = scraper.get(url)
            print(response.status_code, int(get_id_cong_ty(soup)))
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                if get_id_cong_ty(soup):
                    data = {
                        'Url': str(get_url(soup)+ "?fromPage=jobDetail"),
                        'ID': int(get_id_cong_ty(soup)),
                        'Logo': str(get_logo(soup)),
                        'Name': str(get_company_name(soup)),
                        'CompanySize': get_company_info(soup).get('quyMo'),
                        'Field': get_company_info(soup).get('linhVuc'),
                        'Contact': get_company_info(soup).get('lienHe'),
                        'CompanyIntroduction': str(get_gioi_thieu(soup)),
                        'CompanyAddress': str(get_dia_chi(soup)),
                        'Welfare': get_phuc_loi(soup),
                    }
                    output.append(data)
                    print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))
                    print(response.status_code, "Cập nhật thông tin thành công.", int(get_id_cong_ty(soup)), len(output))
            if response.status_code == 429:
                    # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại


200 395774
{
     "Url": "https://www.vietnamworks.com/nha-tuyen-dung/thornton-tomasetti-vietnam-c261563?fromPage=jobDetail",
     "ID": 261563,
     "Logo": "https://images.vietnamworks.com/pictureofcompany/8a/11128523.png",
     "Name": "Thornton Tomasetti Vietnam",
     "CompanySize": null,
     "Field": "Kỹ thuật xây dựng/Cơ sở hạ tầng",
     "Contact": "Kevin Dang",
     "CompanyIntroduction": "Thornton Tomasetti provides engineering design, investigation and analysis services to clients worldwide on projects of every size and level of complexity.  We are a growing 1700+ person firm with 10 practices: Structural Engineering, Forensics, Applied Science, Renewal, Property Loss Consulting, Construction Engineering, Protective Design and Security, Facade Engineering, Sustainability and Transportation. We work on everything from landmark buildings to small-scale specialty structures, from the historic to the high performing, while balancing multiple objectives, including form, function

In [207]:
# # Lưu vào tệp JSON
with open("sample_vietnamworks_company_v4.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, indent=5, default=default_converter)